# Lesson 5: Introduction to excitatory and inhibitory synapses

## Lesson goals:

1. Discuss challenge from last week:
 - Reproduce Figures 2b, 3a, and 3c from Magee & Cook
2. How to activate postsynaptic currents with presynaptic spikes in NEURON: exp2syn, NetCon and VecStim
3. How to compute a synaptic I-V curve for an excitatory synapse (with AMPA-type glutamate receptors).
4. How to populate a dendrite with many synapses.
5. How to construct a random input pattern.
6. How to compute a synaptic input-output curve.
7. Introduction to inhibitory synapses (with GABA(A) receptors).
7. Challenge: 
 - Visualize the impact of inhibitory synaptic input on excitatory input summation.

## 1. Discuss challenge from last week:
### Reproduce Figures 2b, 3a, and 3c from Magee & Cook

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from neuron import h
h.load_file('stdrun.hoc')

Let's construct the same two-compartment neuron as last week, with spiking mechanisms turned off.

In [ ]:
soma = h.Section()
soma.L = 20.  # um
soma.diam = 20.  # um
soma.Ra = 100.  # MOhm

soma.insert('hh')
soma.gnabar_hh = 0.
soma.gkbar_hh = 0.
soma.gl_hh = 4.288E-04
soma.el_hh = -65.

t = h.Vector()
soma_voltage = h.Vector()
t.record(h._ref_t)  # record the time stamp
soma_voltage.record(soma(0.5)._ref_v)  # record the voltage across the membrane in a segment

dend = h.Section()
dend.L = 300.
dend.diam = 1.5
dend.nseg = 9  # Split it into 9 parts to increase the spatial resolution of the simulation
dend.Ra = 100.

# parent section, parent location, child location
dend.connect(soma, 1., 0.)

dend.insert('hh')

# Let's just simulate passive dendrites without Na or K channels for today.
# Here is syntax for setting a membrane density parameter for all segments of a section at once:
dend.gnabar_hh = 0.
dend.gkbar_hh = 0.
dend.gl_hh = 4.288E-04
dend.el_hh = -65.

dend_voltage_rec_list = []
for seg in dend:
    dend_rec = h.Vector()
    dend_rec.record(seg._ref_v)  # record the voltage across the membrane in a segment
    dend_voltage_rec_list.append(dend_rec)

h.tstop = 600.

We need a stimulus to inject current into the dendrite:

In [ ]:
dend_current_stim = h.IClamp(dend(1.))
dend_current_stim.amp = 0.
dend_current_stim.dur = 1e9  # the waveform will determine the duration
dend_current_stim.delay = 0.  # the baseline is now built in to the waveform
h.run()

We want our stimulus to have exponential rise and decay like a synaptic current.

In [ ]:
t_array = np.array(t)
rise_and_decay = np.exp(-(t_array - 200.) / 10.) - np.exp(-(t_array - 200.) / 0.5)
rise_and_decay[np.where(t_array < 200.)[0]] = 0.
rise_and_decay /= np.max(rise_and_decay)
plt.figure()
plt.plot(t_array, rise_and_decay)

We can play our stimulus waveform into our stimulus object:

In [ ]:
dend_stim_waveform_amp = 0.02 # nA
dend_stim_amp_vector = h.Vector(dend_stim_waveform_amp * rise_and_decay)
dend_stim_t_vector = h.Vector(t_array)
dend_stim_amp_vector.play(dend_current_stim._ref_amp, dend_stim_t_vector)

In [ ]:
h.run()

fig, axes = plt.subplots(1, 2, sharex=True)
t_array = np.array(t)
soma_voltage_array = np.array(soma_voltage)
baseline_indexes = np.where((t_array >= 190.) & (t_array < 200.))[0]
EPSP_indexes = np.where((t_array >= 200.) & (t_array < 250.))[0]
baseline = np.mean(soma_voltage_array[baseline_indexes])
soma_voltage_array -= baseline
axes[0].plot(t, soma_voltage_array, label='Soma')
axes[1].plot(t, soma_voltage_array / np.max(soma_voltage_array[EPSP_indexes]))

for dend_voltage, seg in zip(dend_voltage_rec_list, dend):
    distance = dend.L * seg.x
    dend_voltage_array = np.array(dend_voltage)
    baseline = np.mean(dend_voltage_array[baseline_indexes])
    dend_voltage_array -= baseline
    axes[0].plot(t, dend_voltage_array, label='Dend (%.2f um)' % distance)
    axes[1].plot(t, dend_voltage_array / np.max(dend_voltage_array[EPSP_indexes]))

axes[0].legend(loc='best', frameon=False, framealpha=0.5)
axes[0].set_ylabel('Amplitude (mV)')
axes[0].set_xlabel('Time (ms)')
axes[1].set_ylabel('Normalized amplitude (mV)')
axes[0].set_xlim((190., 250.))
axes[0].set_ylim((-0.5, axes[0].get_ylim()[1]))
axes[1].set_ylim((-0.5, axes[1].get_ylim()[1]))

fig.tight_layout()

- Iterate over dendritic segments.
- Move the dendritic stimulus to the new site.
- Record local dendritic and remote somatic voltage.
- Save the data into a sim_history data structure.

In [ ]:
sim_history = []
for i, seg in enumerate(dend):
    sim_record = {}
    sim_record['stim_distance'] = seg.x * dend.L
    
    dend_current_stim = h.IClamp(dend(seg.x))
    dend_current_stim.amp = 0.
    dend_current_stim.dur = 1e9  # the waveform will determine the duration
    dend_current_stim.delay = 0.  # the baseline is now built in to the waveform
    
    dend_stim_waveform_amp = 0.02 # nA
    dend_stim_amp_vector = h.Vector(dend_stim_waveform_amp * rise_and_decay)
    dend_stim_amp_vector.play(dend_current_stim._ref_amp, dend_stim_t_vector)
    
    h.run()
    
    sim_record['t'] = np.array(t)
    sim_record['soma_voltage'] = np.array(soma_voltage)
    sim_record['dend_voltage'] = np.array(dend_voltage_rec_list[i])
        
    sim_history.append(sim_record)

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True)

for sim_record in sim_history:
    t_array = sim_record['t']
    soma_voltage_array = np.copy(sim_record['soma_voltage'])
    distance = sim_record['stim_distance']
    baseline_indexes = np.where((t_array >= 190.) & (t_array < 200.))[0]
    baseline = np.mean(soma_voltage_array[baseline_indexes])
    soma_voltage_array -= baseline
    sim_record['soma_amplitude'] = np.copy(soma_voltage_array)
    axes[0].plot(t, soma_voltage_array, label='%.1f um from soma' % distance)
    
    dend_voltage_array = np.copy(sim_record['dend_voltage'])
    baseline = np.mean(dend_voltage_array[baseline_indexes])
    dend_voltage_array -= baseline
    sim_record['dend_amplitude'] = np.copy(dend_voltage_array)
    axes[1].plot(t, dend_voltage_array)

axes[0].set_title('Soma recording')
axes[1].set_title('Dend recording')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
axes[0].set_ylabel('Amplitude (mV)')
axes[0].set_xlabel('Time (ms)')
axes[1].set_xlabel('Time (ms)')
axes[0].set_xlim((190., 250.))
axes[0].set_ylim((-0.5, axes[1].get_ylim()[1]))
axes[1].set_ylim((-0.5, axes[1].get_ylim()[1]))

fig.tight_layout()

Now let's measure the amplitude and rise time of these events in soma and dendrite:

In [ ]:
soma_rise = []
dend_rise = []
soma_amp = []
dend_amp = []
distances = []

for sim_record in sim_history:
    
    distances.append(sim_record['stim_distance'])
    
    t_array = sim_record['t']
    EPSP_indexes = np.where((t_array >= 200.) & (t_array < 250.))[0]

    soma_amplitude_array = sim_record['soma_amplitude']
    this_soma_amp = np.max(soma_amplitude_array[EPSP_indexes])
    soma_amp.append(this_soma_amp)
    
    soma_peak_index = np.argmax(soma_amplitude_array[EPSP_indexes])
    this_soma_rise = t_array[EPSP_indexes][soma_peak_index] - 200.
    soma_rise.append(this_soma_rise)
    
    dend_amplitude_array = sim_record['dend_amplitude']
    this_dend_amp = np.max(dend_amplitude_array[EPSP_indexes])
    dend_amp.append(this_dend_amp)
    
    dend_peak_index = np.argmax(dend_amplitude_array[EPSP_indexes])
    this_dend_rise = t_array[EPSP_indexes][dend_peak_index] - 200.
    dend_rise.append(this_dend_rise)

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True)

axes[0].plot(distances, soma_amp, label='Soma')
axes[0].plot(distances, dend_amp, label='Dend')
axes[1].plot(distances, soma_rise, label='Soma')
axes[1].plot(distances, dend_rise, label='Dend')

axes[0].set_title('EPSP amplitude')
axes[1].set_title('EPSP rise time')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
axes[1].legend(loc='best', frameon=False, framealpha=0.5)
axes[0].set_ylabel('Amplitude (mV)')
axes[1].set_ylabel('Rise time (ms)')
axes[0].set_xlabel('Distance from soma (um)')
axes[1].set_xlabel('Distance from soma (um)')

fig.tight_layout()

## 2. How to activate postsynaptic currents with presynaptic spikes in NEURON: exp2syn, NetCon and VecStim

Last week we manually constructed an EPSC-shaped current waveform to inject into the dendrite.

Let's turn that off:

In [ ]:
dend_current_stim.dur = 0.

But real synapses are conductance-based, so the current amplitude varies with membrane voltage.

Also, how do we trigger synaptic responses with spikes?

Let's use a synaptic point process that generates a current with exponential rise and decay whenever it is triggered by a spike source:

Let's take a look at the source code in exp2EPSG.mod, and don't forget to complile this new mechanism with `nrnivmodl`

In [ ]:
# Let's insert a synapse in the dendrite ~150 um from the soma:
syn = h.Exp2EPSG(dend(0.5))

In [ ]:
# tab to complete to explore attributes of the synaptic point process object:
syn.

We can control the rise and decay kinetics of this current with the `tau1` and `tau2` attributes:

In [ ]:
syn.tau1 = 0.5
syn.tau2 = 10.

The `h.NetCon` object is used to deliver spikes detected in a presynaptic spike detector to a postsynaptic target point process.

But in this case we don't have a presynaptic cell, we just want to manually define a vector of spike times.

For this we need a special "Artificial Cell" called a `VecStim`

In [ ]:
vs = h.VecStim()

In [ ]:
# tab to complete to explore attributes of the VecStim object:
vs.

# The play() method can be used to load a vector of spike times

Now we can connect the VecStim to our synaptic point process using a `NetCon`:

In [ ]:
nc = h.NetCon(vs, syn)

In [ ]:
# tab to complete to explore attributes of the NetCon object:
nc.

What are the default values of the `delay` and `weight` attributes?

In [ ]:
print('Delay:', nc.delay, 'Weight', nc.weight)

The `delay` attribute is used to emulate axonal conduction delays. For now, we want our spikes to activate synapses without delay, so this can be set to zero.

In [ ]:
nc.delay = 0.

The `weight` attribute of a `NetCon` is actually a vector. For the EPSG mechanism, just the first element in this vector is used, and it controls the amplitude of the postsynaptic current:

In [ ]:
print('Delay:', nc.delay, 'Weight', nc.weight[0])

Let's set the `weight` attribute to be one for now (the synaptic amplitude is also controlled by the `gmax` attribute of the EPSG object).

In [ ]:
nc.weight[0] = 1.

Now we need a spike train to play into our `VecStim` --> `NetCon` --> `EPSG` object chain.

In [ ]:
presyn_spike_times_list = [200.]
vs.play(h.Vector(presyn_spike_times_list))

### Run a simulation and plot the results:

In [ ]:
h.run()

# It's good practice to play an empty vector in to the `VecStim` at the end of every simulation to "reset" it.
vs.play(h.Vector())

In [ ]:
fig, axes = plt.subplots(1, 2)
axes[0].plot(t, soma_voltage)
axes[0].set_title('Soma Vm')
for i, seg in enumerate(dend):
    distance = seg.x * dend.L
    dend_voltage_rec = dend_voltage_rec_list[i]
    axes[1].plot(t, dend_voltage_rec)
axes[1].set_title('Dend Vm')
axes[0].set_ylabel('Voltage (mV)')
axes[0].set_xlabel('Time (ms)')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
fig.tight_layout()
fig.show()

That was way too much current! A single even almost reached the reversal potential of 0 mV!

In [ ]:
exc_syn_gmax = 0.00015
syn.gmax = exc_syn_gmax
vs.play(h.Vector(presyn_spike_times_list))
h.run()
vs.play(h.Vector())

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True)
axes[0].plot(t, soma_voltage)
axes[0].set_title('Soma Vm')
for i, seg in enumerate(dend):
    distance = seg.x * dend.L
    dend_voltage_rec = dend_voltage_rec_list[i]
    axes[1].plot(t, dend_voltage_rec)
axes[1].set_title('Dend Vm')
axes[0].set_ylabel('Voltage (mV)')
axes[0].set_xlabel('Time (ms)')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
axes[0].set_xlim((190., 250.))
fig.tight_layout()
fig.show()

This is better. ~10 pA EPSC and ~0.5 mV EPSP at the soma is more realistic.

## 3. How to compute a synaptic I-V curve for an excitatory synapse (with AMPA-type glutamate receptors).

Now that we're stimulating a conductance-based synapse, the current through the synaptic receptors should depend on the distance between Vm and E, the reversal potential of the receptor. In this case, AMPA-Rs have a reversal potential of 0 mV.

So we'll need to use a somatic step current injection to drive the Vm to different levels, and we'll need to record the current through the receptors.

In [ ]:
step_current_stim = h.IClamp(soma(0.5))
step_current_stim.dur = 600.
step_current_stim.delay = 0.

syn_current = h.Vector()
syn_current.record(syn._ref_i, h.dt)

Now let's iterate over values of step current injection to make sure we're covering values of Vm in a reasonable range from -80 mV to +80 mV:

In [ ]:
plt.figure()
for stim_amp in np.arange(-0.2, 1.6, 0.1):
    step_current_stim.amp = stim_amp
    h.run()
    plt.plot(t, soma_voltage)
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage (mV)')

Now let's stimulate our one synapse on the background of different levels of Vm depolarization:

In [ ]:
# first set the synapse to receive a spike at 200 ms
presyn_spike_times_list = [200.]
vs.play(h.Vector(presyn_spike_times_list))

sim_history = []
for stim_amp in np.arange(-0.2, 1.6, 0.1):
    step_current_stim.amp = stim_amp
    sim_record = {}
    sim_record['stim_amp'] = stim_amp
    
    h.run()
    
    sim_record['t'] = np.array(t)
    sim_record['soma_voltage'] = np.array(soma_voltage)
    sim_record['syn_current'] = np.array(syn_current)
            
    sim_history.append(sim_record)
    
step_current_stim.amp = 0.  # turn off the somatic step current
vs.play(h.Vector())  # reset the synapse to receive zero spikes

Now let's plot the traces and collect the values of Vm and peak synaptic current amplitude:

In [ ]:
soma_Vm = []
syn_current_peak_amp = []

fig, axes = plt.subplots(1, 3, figsize=(10., 4.))
for sim_record in sim_history:
    t_array = sim_record['t']
    soma_voltage_array = sim_record['soma_voltage']
    background_indexes = np.where((t_array >= 190.) & (t_array < 200.))[0]
    this_soma_vm = np.mean(soma_voltage_array[background_indexes])
    soma_Vm.append(this_soma_vm)
    
    syn_current_array = sim_record['syn_current']
    EPSP_indexes = np.where((t_array >= 200.) & (t_array < 250.))[0]
    
    peak_index = np.argmax(np.abs(syn_current_array[EPSP_indexes]))
    this_peak_amp = syn_current_array[EPSP_indexes][peak_index]
    syn_current_peak_amp.append(this_peak_amp)
    
    axes[0].plot(t, soma_voltage_array)
    axes[1].plot(t, syn_current_array)

axes[0].set_ylabel('Voltage (mV)')
axes[0].set_xlabel('Time (ms)')
axes[0].set_xlim((190., 250.))
axes[1].set_ylabel('Peak current amplitude (nA)')
axes[1].set_xlabel('Time (ms)')
axes[1].set_xlim((190., 250.))
axes[2].plot(soma_Vm, syn_current_peak_amp)
axes[2].plot([np.min(soma_Vm), np.max(soma_Vm)], (0., 0.), '--', c='grey')
axes[2].plot((0., 0.), [np.min(syn_current_peak_amp), np.max(syn_current_peak_amp)], '--', c='grey')
axes[2].set_xlabel('Voltage (mV)')
axes[2].set_ylabel('Current (nA)')

fig.tight_layout()

Why didn't this reverse at exactly 0 mV?

## 4. How to populate a dendrite with many synapses.

We'll have to keep a list of all of the objects we need to stimulate 500 synapses:

In [ ]:
num_exc_syns = 500
exc_syn_list = []
exc_vec_stim_list = []
exc_netcon_list = []
exc_syn_locs = []

random_seed = 0
local_random = np.random.RandomState()
local_random.seed(random_seed)

for i in range(num_exc_syns):
    syn_loc = local_random.random()  # random number between 0 and 1
    exc_syn_locs.append(syn_loc)
    
    syn = h.Exp2EPSG(dend(syn_loc))
    syn.gmax = exc_syn_gmax
    exc_syn_list.append(syn)
    
    vs = h.VecStim()
    exc_vec_stim_list.append(vs)
    
    nc = h.NetCon(vs, syn)
    nc.delay = 0.
    nc.weight[0] = 1.
    exc_netcon_list.append(nc)

## 5. How to construct a random input pattern.

Let's stimulate each synapse with just one spike within a 200 ms window.

In [ ]:
exc_syn_spike_time_list = []
for i in range(num_exc_syns):
    spike_times = [local_random.uniform(200., 400.)]
    exc_syn_spike_time_list.append(spike_times)

How let's run 500 simulations, where the number of stimulated inputs increases from 1 to 500!

In [ ]:
sim_history = []
for i in range(num_exc_syns):
    vs = exc_vec_stim_list[i]
    spike_times = exc_syn_spike_time_list[i]
    vs.play(h.Vector(spike_times))
    
    sim_record = {}
    sim_record['num_stim_exc_syns'] = i + 1
    
    h.run()
    
    sim_record['t'] = np.array(t)
    sim_record['soma_voltage'] = np.array(soma_voltage)
    sim_record['dend_voltage'] = np.array(dend_voltage_rec_list[4])  # This dend recording is in the center of the dendrite.
    
    sim_history.append(sim_record)

We should reset all the vec_stims to zero input spikes:

In [ ]:
for i in range(num_exc_syns):
    vs = exc_vec_stim_list[i]
    vs.play(h.Vector())

Let's plot the results!

In [ ]:
import matplotlib.colors

cmap = plt.get_cmap("viridis", num_exc_syns)
norm = matplotlib.colors.Normalize(vmin=0, vmax=num_exc_syns)
sm = plt.cm.ScalarMappable(norm=norm, cmap=cmap)

fig, axes = plt.subplots(1, 2, sharex=True)

for i, sim_record in enumerate(sim_history):
    t_array = sim_record['t']
    soma_voltage_array = np.copy(sim_record['soma_voltage'])
    num_stim_exc_syns = sim_record['num_stim_exc_syns']
    baseline_indexes = np.where((t_array >= 190.) & (t_array < 200.))[0]
    baseline = np.mean(soma_voltage_array[baseline_indexes])
    soma_voltage_array -= baseline
    sim_record['soma_amplitude'] = np.copy(soma_voltage_array)
    axes[0].plot(t, soma_voltage_array, c=cmap(i))
    
    dend_voltage_array = np.copy(sim_record['dend_voltage'])
    baseline = np.mean(dend_voltage_array[baseline_indexes])
    dend_voltage_array -= baseline
    sim_record['dend_amplitude'] = np.copy(dend_voltage_array)
    axes[1].plot(t, dend_voltage_array, c=cmap(i))

cbar = fig.colorbar(sm)
cbar.set_label('Number of exc syns', rotation=270., labelpad=15.)
axes[0].set_title('Soma recording')
axes[1].set_title('Dend recording')
# axes[0].legend(loc='best', frameon=False, framealpha=0.5)
axes[0].set_ylabel('Amplitude (mV)')
axes[0].set_xlabel('Time (ms)')
axes[1].set_xlabel('Time (ms)')
axes[0].set_xlim((190., 450.))
axes[0].set_ylim((-0.5, axes[1].get_ylim()[1]))
axes[1].set_ylim((-0.5, axes[1].get_ylim()[1]))

fig.tight_layout()

## 6. How to compute a synaptic input-output curve.

In [ ]:
mean_soma_vm = []
mean_dend_vm = []
num_stim_exc_syns = []

for sim_record in sim_history:
    t_array = sim_record['t']
    soma_voltage_array = sim_record['soma_amplitude']
    stim_indexes = np.where((t_array >= 200.) & (t_array < 400.))[0]
    this_mean_soma_vm = np.mean(soma_voltage_array[stim_indexes])
    mean_soma_vm.append(this_mean_soma_vm)
    
    dend_voltage_array = sim_record['dend_amplitude']
    this_mean_dend_vm = np.mean(dend_voltage_array[stim_indexes])
    mean_dend_vm.append(this_mean_dend_vm)
    
    num_stim_exc_syns.append(sim_record['num_stim_exc_syns'])

In [ ]:
plt.figure()
plt.plot(num_stim_exc_syns, mean_soma_vm, label='Soma amp')
plt.plot(num_stim_exc_syns, mean_dend_vm, label='Dend amp')
plt.legend(loc='best', frameon=False)
plt.xlabel('Number of stimulated exc synapses')
plt.ylabel('Amplitude (mV)')

## 7. Introduction to inhibitory synapses (with GABA(A) receptors).

In [ ]:
i_syn = h.Exp2EPSG(dend(0.5))
i_syn.tau1 = 0.5
i_syn.tau2 = 10.
i_syn.e = -70.  # mV GABA(A) receptors have a reversal potential of ~-70 mV
i_syn_gmax = exc_syn_gmax

i_syn_current = h.Vector()
i_syn_current.record(i_syn._ref_i, h.dt)

i_vs = h.VecStim()
i_nc = h.NetCon(i_vs, i_syn)
i_nc.delay = 0.
i_nc.weight[0] = 1.

We can compute an I-V curve for this inhibitory synaptic receptor, too.

In [ ]:
# first set the synapse to receive a spike at 200 ms
presyn_spike_times_list = [200.]
i_vs.play(h.Vector(presyn_spike_times_list))

sim_history = []
for stim_amp in np.arange(-0.2, 1.6, 0.1):
    step_current_stim.amp = stim_amp
    sim_record = {}
    sim_record['stim_amp'] = stim_amp
    
    h.run()
    
    sim_record['t'] = np.array(t)
    sim_record['soma_voltage'] = np.array(soma_voltage)
    sim_record['i_syn_current'] = np.array(i_syn_current)
            
    sim_history.append(sim_record)
    
step_current_stim.amp = 0.  # turn off the somatic step current
i_vs.play(h.Vector())  # reset the synapse to receive zero spikes

In [ ]:
soma_Vm = []
syn_current_peak_amp = []

fig, axes = plt.subplots(1, 3, figsize=(10., 4.))
for sim_record in sim_history:
    t_array = sim_record['t']
    soma_voltage_array = sim_record['soma_voltage']
    background_indexes = np.where((t_array >= 190.) & (t_array < 200.))[0]
    this_soma_vm = np.mean(soma_voltage_array[background_indexes])
    soma_Vm.append(this_soma_vm)
    
    syn_current_array = sim_record['i_syn_current']
    EPSP_indexes = np.where((t_array >= 200.) & (t_array < 250.))[0]
    
    peak_index = np.argmax(np.abs(syn_current_array[EPSP_indexes]))
    this_peak_amp = syn_current_array[EPSP_indexes][peak_index]
    syn_current_peak_amp.append(this_peak_amp)
    
    axes[0].plot(t, soma_voltage_array)
    axes[1].plot(t, syn_current_array)

axes[0].set_ylabel('Voltage (mV)')
axes[0].set_xlabel('Time (ms)')
axes[0].set_xlim((190., 300.))
axes[1].set_ylabel('Peak current amplitude (nA)')
axes[1].set_xlabel('Time (ms)')
axes[1].set_xlim((190., 300.))
axes[2].plot(soma_Vm, syn_current_peak_amp)
axes[2].plot([np.min(soma_Vm), np.max(soma_Vm)], (0., 0.), '--', c='grey')
axes[2].plot((-70., -70.), [np.min(syn_current_peak_amp), np.max(syn_current_peak_amp)], '--', c='grey')
axes[2].set_xlabel('Voltage (mV)')
axes[2].set_ylabel('Current (nA)')

fig.tight_layout()

## 8. Challenge

### Compute the effect of activating increasing numbers of inhibitory synapses on the synaptic input-output curve.